In [1]:
# Necessary when connecting to a jupyterhub kernel running on daint via VScode. NOT required otherwise

import os

new_path = '/users/ajayaraj/scratch/tests/qtpyt-tests/AuBDA13CH2'
os.chdir(new_path)


In [2]:
from pathlib import Path
import numpy as np
from gpaw import restart
from gpaw.lcao.pwf2 import LCAOwrap
from gpaw.lcao.tools import remove_pbc
from matplotlib.colors import LinearSegmentedColormap, LogNorm
from qtpyt.basis import Basis
from qtpyt.lo.tools import rotate_matrix, subdiagonalize_atoms,cutcoupling
from ase.io import read
from qtpyt.basis import Basis
from qtpyt.block_tridiag import graph_partition
from qtpyt.tools import remove_pbc
from copy import deepcopy


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


### Helper functions

In [3]:
def get_species_indices(atoms,species):
    indices = []
    for element in species:
        element_indices = atoms.symbols.search(element)
        indices.extend(element_indices)
    return sorted(indices)


In [4]:
def create_active_hamiltonian(H_cutcoupled, S_cutcoupled, index_left_Au, index_right_Au,  index_active_region):
    """
    Create the Hamiltonian H_active by retaining the left and right Au orbitals and
    the active region couplings within the bridge.

    Parameters:
    H_cutcoupled (np.ndarray): The Hamiltonian matrix after cutting couplings.
    index_active_region (list of int): Indices of the active orbitals within the bridge.

    Returns:
    np.ndarray: The Hamiltonian matrix H_active.
    """

    # Create a list of indices to retain
    indices_to_keep = index_left_Au + sorted(list(index_active_region)) + index_right_Au

    # Create the H_active matrix by extracting the relevant rows and columns
    H_active = H_cutcoupled[np.ix_(indices_to_keep, indices_to_keep)]
    S_active = S_cutcoupled[np.ix_(indices_to_keep, indices_to_keep)]

    return H_active[None,...], S_active[None,...]


### Control parameters

In [5]:
GPWDEVICEDIR = 'dft/device/'
BRIDGE_SPECIES = ("N", "C", "H")
GPWLEADSDIR = 'dft/leads/'

In [6]:
lowdin = True
cc_path = Path(GPWDEVICEDIR)
pl_path = Path(GPWLEADSDIR)
gpwfile = f'{cc_path}/scatt.gpw'

atoms, calc = restart(gpwfile, txt=None)
fermi = calc.get_fermi_level()
nao_a = np.array([setup.nao for setup in calc.wfs.setups])
basis = Basis(atoms, nao_a)

H_lcao, S_lcao = np.load("dft/device/hs_cc_k.npy")


In [7]:
S_lcao.shape

(1, 2394, 2394)

In [8]:
cmap_name = 'custom_white_red'
colors = [(1, 1, 1), (166/255, 4/255, 4/255)]
n_bins = 100
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
norm = LogNorm(vmin=0.1, vmax=10)

In [9]:
H_leads_lcao, S_leads_lcao = np.load(pl_path / 'hs_pl_k.npy')

basis_dict = {'Au': 9, 'H': 5, 'C': 13, 'N': 13}

leads_atoms = read(pl_path / 'leads.xyz')
leads_basis = Basis.from_dictionary(leads_atoms, basis_dict)

device_atoms = read(cc_path / 'scatt.xyz')
device_basis = Basis.from_dictionary(device_atoms, basis_dict)

cutoff = [1,1e-2,1e-4,1e-5,1e-6,1e-8,1e-10,1e-18,1e-40]
# Define the number of repetitions (Nr) and unit cell repetition in the leads
Nr = (1, 5, 3)

### Nodes obtained from LCAO Hamiltonian (Hamiltonian obtained directly from gpaw)

In [10]:
# Remove periodic boundary conditions (PBC) from the device Hamiltonian and overlap matrices
remove_pbc(device_basis, H_lcao)
remove_pbc(device_basis, S_lcao)


for co in cutoff:
    # Partition the device Hamiltonian matrix into tridiagonal nodes
    nodes = graph_partition.get_tridiagonal_nodes(device_basis, H_lcao[0], len(leads_atoms.repeat(Nr)))

    # Print the nodes to verify the partitioning
    print(nodes)


[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]


### Nodes obtained from Hamiltonian that Subdiagonalizes C, N and H

In [11]:
# Perform subdiagonalization
SUBDIAG_SPECIES = ("C", "N", "H")
subdiag_indices = get_species_indices(atoms, SUBDIAG_SPECIES)
Usub, eig = subdiagonalize_atoms(basis, H_lcao[0], S_lcao[0], a=subdiag_indices)

# Rotate matrices
H_sub = rotate_matrix(H_lcao[0], Usub)[None, ...]
S_sub = rotate_matrix(S_lcao[0], Usub)[None, ...]

# Define the number of repetitions (Nr) and unit cell repetition in the leads
Nr = (1, 5, 3)

# Remove periodic boundary conditions (PBC) from the device Hamiltonian and overlap matrices
remove_pbc(device_basis, H_sub)
remove_pbc(device_basis, S_sub)
for co in cutoff:
    # Partition the device Hamiltonian matrix into tridiagonal nodes
    nodes = graph_partition.get_tridiagonal_nodes(device_basis, H_sub[0], len(leads_atoms.repeat(Nr)))

    print(nodes)


[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]


### Nodes obtained from Hamiltonian that Subdiagonalizes only C

In [16]:
# Perform subdiagonalization
SUBDIAG_SPECIES = ("C")
subdiag_indices = get_species_indices(atoms, SUBDIAG_SPECIES)
Usub, eig = subdiagonalize_atoms(basis, H_lcao[0], S_lcao[0], a=subdiag_indices)

# Rotate matrices
H_sub = rotate_matrix(H_lcao[0], Usub)[None, ...]
S_sub = rotate_matrix(S_lcao[0], Usub)[None, ...]

# Define the number of repetitions (Nr) and unit cell repetition in the leads
Nr = (1, 5, 3)

# Remove periodic boundary conditions (PBC) from the device Hamiltonian and overlap matrices
remove_pbc(device_basis, H_sub)
remove_pbc(device_basis, S_sub)

for co in cutoff:
    # Partition the device Hamiltonian matrix into tridiagonal nodes
    nodes = graph_partition.get_tridiagonal_nodes(device_basis, H_sub[0], len(leads_atoms.repeat(Nr)),cutoff=co)

    # Print the nodes to verify the partitioning
    print(nodes)


[0, 2394]
[0, 810, 1107, 1170, 1224, 1287, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1278, 1584, 2394]
[0, 810, 1116, 1278, 1584, 2394]
[0, 810, 1121, 1273, 1584, 2394]
[0, 810, 1144, 1250, 1584, 2394]
[0, 810, 1144, 1224, 1584, 2394]
[0, 810, 1144, 1224, 1584, 2394]


### Nodes obtained from Hamiltonian that Subdiagonalizes only H

In [17]:
# Perform subdiagonalization
SUBDIAG_SPECIES = ("H")
subdiag_indices = get_species_indices(atoms, SUBDIAG_SPECIES)
Usub, eig = subdiagonalize_atoms(basis, H_lcao[0], S_lcao[0], a=subdiag_indices)

# Rotate matrices
H_sub = rotate_matrix(H_lcao[0], Usub)[None, ...]
S_sub = rotate_matrix(S_lcao[0], Usub)[None, ...]

# Define the number of repetitions (Nr) and unit cell repetition in the leads
Nr = (1, 5, 3)

# Remove periodic boundary conditions (PBC) from the device Hamiltonian and overlap matrices
remove_pbc(device_basis, H_sub)
remove_pbc(device_basis, S_sub)

for co in cutoff:
    # Partition the device Hamiltonian matrix into tridiagonal nodes
    nodes = graph_partition.get_tridiagonal_nodes(device_basis, H_sub[0], len(leads_atoms.repeat(Nr)),cutoff=co)

    # Print the nodes to verify the partitioning
    print(nodes)


[0, 2394]
[0, 810, 1107, 1170, 1250, 1287, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1278, 1584, 2394]
[0, 810, 1121, 1278, 1584, 2394]
[0, 810, 1121, 1273, 1584, 2394]
[0, 810, 1144, 1250, 1584, 2394]
[0, 810, 1144, 1224, 1584, 2394]
[0, 810, 1144, 1224, 1584, 2394]


### Nodes obtained from Hamiltonian that Subdiagonalizes only N

In [18]:
# Perform subdiagonalization
SUBDIAG_SPECIES = ("N")
subdiag_indices = get_species_indices(atoms, SUBDIAG_SPECIES)
Usub, eig = subdiagonalize_atoms(basis, H_lcao[0], S_lcao[0], a=subdiag_indices)

# Rotate matrices
H_sub = rotate_matrix(H_lcao[0], Usub)[None, ...]
S_sub = rotate_matrix(S_lcao[0], Usub)[None, ...]

# Define the number of repetitions (Nr) and unit cell repetition in the leads
Nr = (1, 5, 3)

# Remove periodic boundary conditions (PBC) from the device Hamiltonian and overlap matrices
remove_pbc(device_basis, H_sub)
remove_pbc(device_basis, S_sub)


for co in cutoff:
    # Partition the device Hamiltonian matrix into tridiagonal nodes
    nodes = graph_partition.get_tridiagonal_nodes(device_basis, H_sub[0], len(leads_atoms.repeat(Nr)),cutoff=co)

    # Print the nodes to verify the partitioning
    print(nodes)


[0, 2394]
[0, 810, 1107, 1170, 1250, 1287, 1584, 2394]
[0, 810, 1116, 1224, 1278, 1584, 2394]
[0, 810, 1116, 1278, 1584, 2394]
[0, 810, 1116, 1278, 1584, 2394]
[0, 810, 1134, 1273, 1584, 2394]
[0, 810, 1144, 1250, 1584, 2394]
[0, 810, 1144, 1224, 1584, 2394]
[0, 810, 1144, 1224, 1584, 2394]


### Analysis

If we are to assume that the nodes that will give us the right transmission function, are the ones that divide the Hamiltonian indices into [0,Au lead left, Au tip left, molecule, Au lead right, Au tip right], the right nodes would be

nodes = [0,810,1116,1278,1584,2394]


### Conclusion 
The current implementation of `graph_partition.get_tridiagonal_nodes` is unreliable in producing the right nodes. So proceed by hardcoding the nodes.